## <2-3 어휘 집합 구축하기> 

지금까지 살펴본 내용을 바탕으로 허깅페이스 tokenizers 라이브러리를 활용해 BPE 기반의 토크나이저를 만들어 보겠습니다. BPE는 학습 대상 말뭉치에 자주 등장하는 문자열을 토큰으로 인식해, 이를 기반으로 토큰화를 수행하는 기법입니다. BPE 기반 토크나이저를 사용하려면 어휘 집합부터 구축해야 합니다.  

### BPE 기반 토크나이저 만들기
---
**1단계 실습 환경 만들기**

In [ ]:
#의존성 패키지 설치
!pip install ratsnlp

     |████████████████████████████████| 42 kB 1.0 MB/s 
     |████████████████████████████████| 57 kB 5.7 MB/s 
     |████████████████████████████████| 806 kB 15.8 MB/s 
     |████████████████████████████████| 2.8 MB 49.3 MB/s 
     |████████████████████████████████| 134 kB 65.7 MB/s 
     |████████████████████████████████| 829 kB 54.1 MB/s 
     |████████████████████████████████| 636 kB 61.0 MB/s 
     |████████████████████████████████| 397 kB 59.8 MB/s 
     |████████████████████████████████| 67 kB 4.6 MB/s 
     |████████████████████████████████| 895 kB 52.0 MB/s 
     |████████████████████████████████| 3.3 MB 33.2 MB/s 
     |████████████████████████████████| 1.1 MB 45.0 MB/s 
     |████████████████████████████████| 96 kB 6.3 MB/s 
     |████████████████████████████████| 144 kB 77.4 MB/s 
     |████████████████████████████████| 271 kB 50.2 MB/s 
     |████████████████████████████████| 94 kB 3.6 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sh

**2단계 구글 드라이브 연동하기**  

코랩 노트북은 일정 시간 사용하지 않으면 모든 결과물이 날아갈 수 있습니다. 이번 실습에서 구축할 어휘 집합을 따로 저장해 두기 위헤 자신의 구글 드라이브를 코랩 노트북과 연결합니다.

In [21]:
#구글 드라이브와 연결
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


**3단계 말뭉치 내려받기 및 전처리**  
오픈소스 파이썬 패키지 코포라를 활용해 BPE 수행 대상 말뭉치를 내려받고 전처리합니다. 실습용 말뭉치는 박은정 님이 공개한 '네이버 영화 리뷰 NSMC'입니다. 다음 코드를 수행하면 데이터를 내려받아 해당 데이터를 nsmc라는 변수로 읽어들입니다.

In [ ]:
#NSMC 다운로드
from Korpora import Korpora
nsmc = Korpora.load("nsmc", force_download=True)


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/



[nsmc] download ratings_train.txt: 14.6MB [00:00, 117MB/s]                             
[nsmc] download ratings_test.txt: 4.90MB [00:00, 50.8MB/s]


다음 코드를 수행하면 NSMC에 포함된 영화 리뷰들은 순수 텍스트 형태로 지정된 디렉터리에 저장해 둡니다.

In [ ]:
#NSMC 전처리
import os
def write_lines(path, lines):
  with open(path, 'w', encoding='utf-8') as f:
    for line in lines:
      f.write(f'{line}\n')
write_lines("/root/train.txt", nsmc.train.get_all_texts())
write_lines("/root/test.txt", nsmc.test.get_all_texts())

**4단계 GPT 토크나이저 구축**
GPT 계열 모델이 사용하는 토크나이저 기법은 BPE입니다. 단, 앞 절에서 설명한 문자 단위가 아니라 ***유니코드 바이트*** 수준으로 어휘 집합을 구축하고 토큰화를 수행합니다. 전 세계 대부분의 글자는 유니코드로 표현할 수 있으므로 유니코드 바이트 기준 BPE를 사용하면 미등록 토큰 문제에서 비교적 자유롭습니다.  

한글은 한 글자가 3개의 유니코드 바이트로 표현되는데요, 예를 들어 ***안녕하세요*** 라는 문자열을 유니코드 바이트로 변환하면 다음처럼 됩니다.

> **안녕하세요 ➔ %uC548%uB155%uD558%uC138%uC694**

바이트 수준으로 BPE를 수행한다는 것은 어휘 집합 구축 대상 말뭉치를 위와 같이 모두 유니코드 바이트로 변환하고 이들을 문자 취급해 가장 자주 등장한 문자열을 병합하는 방식으로 어휘 집합을 만든다는 의미입니다. 토큰화 역시 문자열을 유니코드 바이트로 일단 변환한 뒤 수행합니다.  

우선 바이트 수준 BPE 어휘 집합 구축 결과를 저장해 둘 디텍터리를 자신의 구글 드라이브 계정 내에 만듭니다. 다음 코드를 실행하면 됩니다.

In [23]:
#디렉터리 만들기
import os
os.makedirs("'/content/drive/My Drive/nlpbook/bbpe", exist_ok=True)

In [24]:
#바이트 수준 BPE 어휘 집합 구축
from tokenizers import ByteLevelBPETokenizer
bytebpe_tokenizer = ByteLevelBPETokenizer()
bytebpe_tokenizer.train(
    files=["/content/drive/train.txt", "/content/drive/test.txt"],
    vocab_size=10000,
    special_tokens=["[PAD]"]
)
bytebpe_tokenizer.save_model("/content/drive/My Drive/nlpbook/bbpe")

Exception: ignored

코드 수행이 끝나면 /gdrive/My Drive/nlpbook/bbpe 디렉터리에 vocab.json과 merges.txt가 생성됩니다. 전자는 바이트 수준 BPE의 어휘 집합이며 후자는 바이그램 쌍의 병합 우선 순위입니다. 각각의 역할은 이전에 내용들을 참고해 주세요.  
다음은 수행 결과 일부를 보여줍니다.

**5단계 BERT 토크나이저 구축**  

BERT는 워드피스 토크나이저를 사용합니다. 먼저 앞에서 살펴본 코드로 자신의 구글 드라이브를 연결하고, 또 코드 실행을 통해 학습 말뭉치를 준비합니다. 그다음으로 워드피스 어휘 집합 구축 결과를 저장해 둘 디렉터리를 자신의 구글 드라이브 계정 내에 만듭니다.

In [19]:
#디렉터리 만들기
import os
os.makedirs("/gdrive/My Drive/nlpbook/wordpiece", exist_ok=True)

이후 다음 코드를 실행하면 BERT 모델이 사용하는 워드피스 어휘 집합을 구축할 수 있습니다.

In [20]:
#워드피스 어휘 집합 구축
from tokenizers import BertWordPieceTokenizer
wordpiece_tokenizer = BertWordPieceTokenizer(lowercase=False)
wordpiece_tokenizer.train(
    files=["/content/train.txt", "/content/test.txt"],
    vocab_size=10000,
)
wordpiece_tokenizer.save_model("/gdrive/My Drive/nlpbook/wordpiece")

Exception: ignored

코드 수행이 끝나면 save_path가 가리키는 위치에 워드피스 어휘 집합인 vocab.txt가 생성됩니다.